In [37]:
from main import gps_data_utils

import pandas as pd
from datetime import datetime, timedelta

import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic

from collections import defaultdict
import json

In [38]:
df = pd.read_excel('Report_JR14_Vehicle History Report - data.xlsx',skiprows=10)

In [39]:
gdf = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df['Latitude'],df['Longitude']), crs='EPSG:4326')
df = pd.DataFrame(gdf)
#print(df)

In [40]:
def convert_to_gtfs_time(ts,st_ts):
    ts_dt = datetime.strptime(ts, "%Y-%m-%dT%H:%M:%S.%f")
    start_trip_dt = pd.to_datetime(st_ts)
    
    ts_dt_obj = ts_dt.date()
    start_trip_dt_obj = start_trip_dt.date()

    if ts_dt_obj == start_trip_dt_obj:
        return ((ts_dt.hour * 60) + ts_dt.minute + ts_dt.second)
    elif (ts_dt - start_trip_dt).days > 0:
        return (((ts_dt - start_trip_dt).seconds) // 60 + ((ts_dt - start_trip_dt).days * 24 * 60 * 60) // 60)
    else:
        print("Error in Timestamp")
        return None

In [42]:
def calculate_distance(row, prev_row):
    point_a = (row['geometry'].x, row['geometry'].y)
    point_b = (prev_row['geometry'].x, prev_row['geometry'].y)
    return geodesic(point_a, point_b).kilometers

In [ ]:
p1 = Point(19.4230387681323, -99.1283375653607)
p2 = Point(19.4226931599795, -99.1282858845984)
p3 = Point(19.4223342584274, -99.128116747558)
p4 = Point(19.4215765747699, -99.1277173962126)
p5 = Point(19.4212024412726, -99.1275371303434)
p6 = Point(19.4204750900101, -99.1271367956332)
p7 = Point(19.4203918053066, -99.1270190501302)
p8 = Point(19.4174929683974, -99.1274678598945)
p9 = Point(19.4172999746193, -99.1275045888836)
p10 = Point(19.4176612061906, -99.1318657926695)
p11 = Point(19.4179003490392, -99.1319983339035)
p12 = Point(19.4237439344618, -99.1310950549938)
p13 = Point(19.4233598470446, -99.1283536711016)

points = [p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p1]
depot = gps_data_utils.set_depot_boundary(points,buffer = 0)

df = gps_data_utils.check_veh_within_depot(df,depot)

In [43]:
# session_starts = []

current_session_start = None
on_odometer = None
off_odometer = None

trip_ends = defaultdict(list)
calendar = defaultdict(list)
stops = defaultdict(list) # IMPORTANT
stop_times = defaultdict(list) # IMPORTANT
trips = defaultdict(list) # IMPORTANT
routes = defaultdict(list) # IMPORTANT
shapes = defaultdict(list) 
feed_info = defaultdict(list)
agency = defaultdict(list)

blocks_for_input = {"blocks":[]}

df = df[df['Reason'].isin(['Arr. Way Point','Break End','Break Start','Dep. Way Point','Logoff','Logon','Status Report','Ignition Off','Ignition On','Odometer Offset'])]

agency["agency_id"].append("MGL")
agency["agency_name"].append("Microgrid Labs")
agency["agency_url"].append("https://www.microgridlabs.com")
agency["agency_timezone"].append("Asia/Kolkata")

calendar["start_date"].append(pd.to_datetime(df['GPS'].min()).strftime('%Y%m%d'))
calendar["end_date"].append(pd.to_datetime(df['GPS'].max()).strftime('%Y%m%d'))
calendar["monday"].append("1")
calendar["tuesday"].append("1")
calendar["wednesday"].append("1")
calendar["thursday"].append("1")
calendar["friday"].append("1")
calendar["saturday"].append("1")
calendar["sunday"].append("1")
calendar["service_id"].append("S1")

feed_info["feed_publisher_name"].append("Microgrid Labs")
feed_info["feed_publisher_url"].append("https://www.microgridlabs.com")
feed_info["feed_lang"].append("eng")
feed_info["feed_start_date"].append("20230101")
feed_info["feed_end_date"].append("20250101")
feed_info["feed_version"].append("1")
feed_info["feed_url"].append("https://www.microgridlabs.com")

trip_no = 1
block_id = 1
bid = 0
tid = 0
block_changed = True
blocks_data = []
total_block_distance = 0
block_end_details = []

HVAC_energy = 0.0 
Aux_energy = 0.0
stop_portion = 0.5
payload = 0.5

prev_start , prev_stop = None, None
for index, row in df.iterrows():
    if row['Reason'] == 'Ignition On':
        current_session_start = index
        on_odometer = row['Odometer']
    elif row['Reason'] == 'Ignition Off' and current_session_start is not None:
        off_odometer = row['Odometer']

        if on_odometer is not None and off_odometer is not None:
            # Set another logic for selecting trips since SaaS only takes trips < 1km in distance for REM
            diff = off_odometer - on_odometer
            if diff > 1:

                trips_df = df.loc[current_session_start:index].reset_index()

                trips_df['Distance'] = 0.0 

                # Set Trips file here
                trips["trip_id"].append(f"T_{trip_no}")
                trips["route_id"].append(f"R_{trip_no}")
                trips["service_id"].append("S1")

                trips["shape_id"].append(f"Sh_{trip_no}")
                trips["trip_short_name"].append(f"Short Name {trip_no}")
                trips["direction_id"].append("1")
                
                # Set Block Logic here
                # Block logic can also be setup using df.[prev_start and prev_stop] and df.[current_session_start and index]
                if trips_df.iloc[0]['indepot'] == True and (prev_start is not None and prev_stop is not None) and (prev_stop <= len(df)):
                    curr_t_start = df.iloc[current_session_start]['GPS']
                    prev_t_end = df.iloc[prev_stop]['GPS']
                    
                    if (curr_t_start - prev_t_end) < timedelta(minutes=30) and curr_t_start >= prev_t_end:
                        block_id = block_id
                    else:
                        bid+=1
                        prev_block_id = f"B{block_id}"
                        end_time = trip["end_time"]
                        block_end_details.append([prev_block_id, end_time, total_block_distance])
                        block_id+=1
                        block_changed = True
                        total_block_distance = 0 # total_block_distance will become zero when ever we add a new block id 
                        tid = 0
                        #print(total_block_distance)

                trips["block_id"].append(f"B{block_id}")
                # Set Routes file here
                routes["agency_id"].append("MGL")
                routes["route_id"].append(f"R_{trip_no}")
                routes["route_short_name"].append(f"RSn {trip_no}")
                routes["route_long_name"].append(f"Long Name {trip_no}")
                routes["route_type"].append(3)

                prev_start = current_session_start
                prev_stop = index

                trip_start_day = trips_df['GPS'][0].strftime('%Y%m%d')

                # To get the start and end time for each trip and to get the start time for each block id
                start_date_time = trips_df['GPS'].iloc[0]
                end_date_time = trips_df['GPS'].iloc[-1]
                start_time = convert_to_gtfs_time(start_date_time.strftime('%Y-%m-%dT%H:%M:%S.%f'), trip_start_day) 
                end_time = convert_to_gtfs_time(end_date_time.strftime('%Y-%m-%dT%H:%M:%S.%f'), trip_start_day)

                # To get start and end latitude and start and end longitude for each trip and to get the start latitude and start longitude for each block id 
                start_lat = trips_df['Latitude'].iloc[0]
                start_lon = trips_df['Longitude'].iloc[0]
                end_lat = trips_df['Latitude'].iloc[-1]
                end_lon = trips_df['Longitude'].iloc[-1]

                if block_changed:
                    block = {
                        "id": bid,
                        "blockId": f"B{block_id}",
                        "dh_st_time": int(start_time),  
                        "startTime": int(start_time),  
                        "endTime": 0,  
                        "dh_end_time": 0,  
                        "lon_start": float(start_lon),  
                        "lat_start": float(start_lat),  
                        "distance": 0,  
                        "vehicleEff": "",  
                        "vehicleModel": "",  
                        "trips": []
                    }

                trip = {
                    "id": tid,
                    "trip_id": f"T_{trip_no}",
                    "type": 3,
                    "start_time": int(start_time),
                    "start_lat": float(start_lat),
                    "start_lon": float(start_lon),
                    "end_time": int(end_time),
                    "end_lat": float(end_lat),
                    "end_lon": float(end_lon),
                    "distance": 0,
                    "eff": "",
                    "direction": 1,
                    "stops": [],
                    "route_id": f"R_{trip_no}",
                    "route_name": f"RSn {trip_no}",
                    "shape_id": f"Sh_{trip_no}",
                    "HVAC_energy": float(HVAC_energy),  
                    "Aux_energy": float(Aux_energy),  
                    "stop_portion": float(stop_portion),  
                    "payload": float(payload)  
                }

                sid = 0
                for id,trips_row in trips_df.iterrows():  

                    next_id = id + 1 
                    if id < (len(trips_df) - 1):
                        next_row = trips_df.loc[next_id]
                        distance = calculate_distance(trips_row, next_row)
                        trips_df.at[id, 'Distance'] = distance

                    stop_id = round(abs(float(trips_row['Latitude'])),2).__str__() + round(abs(float(trips_row['Longitude'])),2).__str__() + pd.to_datetime(trips_row['GPS']).strftime('%Y%m%d%H%M%S') + id.__str__() 

                    # Set Stops File - DONE
                    if id in [0, len(trips_df) - 1]:
                        stops["stop_id"].append(f"{stop_id}")
                        stops["stop_name"].append(f"Stop {id}")
                        stops["stop_lat"].append(trips_row['Latitude'])
                        stops["stop_lon"].append(trips_row['Longitude'])

                        # Set Stop Times File - DONE
                        stop_times["stop_id"].append(f"{stop_id}")
                        stop_times["trip_id"].append(f"T_{trip_no}")

                        # Timepoint is required for start_time and end_time
                        stop_times["timepoint"].append("1")
                        if id == 0:
                            stop_times["stop_sequence"].append("1")
                        else:
                            stop_times["stop_sequence"].append("2")

                        stop_times["arrival_time"].append(f"{convert_to_gtfs_time(trips_row['GPS'].strftime('%Y-%m-%dT%H:%M:%S.%f'),trip_start_day)}")
                        stop_times["departure_time"].append(f"{convert_to_gtfs_time(trips_row['GPS'].strftime('%Y-%m-%dT%H:%M:%S.%f'),trip_start_day)}")

                        stop = {
                            "id": sid,
                            "stop_id": f"{stop_id}",
                            "arrival_time": convert_to_gtfs_time(trips_row['GPS'].strftime('%Y-%m-%dT%H:%M:%S.%f'), trip_start_day),
                            "departure_time": convert_to_gtfs_time(trips_row['GPS'].strftime('%Y-%m-%dT%H:%M:%S.%f'), trip_start_day),
                            "lat": float(trips_row['Latitude']),
                            "lon": float(trips_row['Longitude'])
                        }
                        
                        trip["stops"].append(stop)

                        sid += 1
                        
                    # Set Shapes File - DONE
                    shapes["shape_id"].append(f"Sh_{trip_no}")
                    shapes["shape_pt_lat"].append(trips_row['Latitude'])
                    shapes["shape_pt_lon"].append(trips_row['Longitude'])
                    shapes["shape_pt_sequence"].append(f"{int(id)+1}")

                total_trip_distance = trips_df['Distance'].sum()
                #print(total_trip_distance)

                trip["distance"] = float(total_trip_distance)

                total_block_distance +=  total_trip_distance 
                #print(total_block_distance)

                # Proceed to next Trip ID
                trip_no += 1
                tid += 1

                block["trips"].append(trip)

                if block_changed:
                    blocks_data.append(block)
                    
                block_changed = False

        else:
            # Do something when the Ignition On or Off row does not have any odometer value
            print("Ignition On or Off do not have an Odometer Value")

json_data = {
    "blocks": blocks_data
}

block_end_details_df = pd.DataFrame(block_end_details, columns=['blockId', 'endTime', 'distance'])

for block in json_data["blocks"]:
    blockId = block['blockId']
    if blockId in block_end_details_df['blockId'].values:
        row = block_end_details_df[block_end_details_df['blockId'] == blockId]
        end_time = row['endTime'].values[0]
        distance = row['distance'].values[0]
        block['endTime'] = int(end_time)
        block['dh_end_time'] = int(end_time)
        block['distance'] = float(distance)

for block in json_data["blocks"]:
    block["startDepot"] = 1  
    block["endDepot"] = 1  
    block["cords"] = []
    for trip in block["trips"]:
        for stop in trip["stops"]:
            coordinates = [float(stop["lon"]), float(stop["lat"])]
            block["cords"].append(coordinates)

In [44]:
if len(stops["stop_id"]) == len(set(stops["stop_id"])):
    print("Y")
else:
    print(len(stops["stop_id"]))
    print(len(set(stops["stop_id"])))

Y


In [45]:
temp_dir = 'output'
pd.DataFrame(agency).to_csv(f"{temp_dir}/agency.txt",index=False)
pd.DataFrame(feed_info).to_csv(f"{temp_dir}/feed_info.txt",index=False)
pd.DataFrame(calendar).to_csv(f"{temp_dir}/calendar.txt",index=False)
pd.DataFrame(stops).to_csv(f"{temp_dir}/stops.txt",index=False)
pd.DataFrame(stop_times).to_csv(f"{temp_dir}/stop_times.txt",index=False)
pd.DataFrame(trips).to_csv(f"{temp_dir}/trips.txt",index=False)
pd.DataFrame(routes).to_csv(f"{temp_dir}/routes.txt",index=False)
pd.DataFrame(shapes).to_csv(f"{temp_dir}/shapes.txt",index=False)

with open(f"{temp_dir}/output.json", 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

# with zipfile.ZipFile("gtfs_output.zip", "w") as zip_file:
#     for filename in ["agency.txt","feed_info.txt","stop_times.txt", "stops.txt", "trips.txt", "routes.txt","calendar.txt","shapes.txt"]:
#         zip_file.write(f"{temp_dir}/{filename}", arcname=filename)